## Multimodel Integration with Guardrails

In this lab, we will explore how to integrate **multiple language models** within an agentic AI framework while implementing guardrails to ensure safe and appropriate outputs. We will use different models for various tasks and set up guardrails to monitor and control the agent's behavior.

Highlights of this lab include:
1. Different models: using OpenAI compatiible endpoints
2. Structured Outputs: using Pydantic with Guardrails
3. Guardrails: setting up input guardrails to prevent sensitive information in outputs
4. Tracing and Monitoring: using OpenAI traces to monitor guardrail activations


## Setup

In [1]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel

In [2]:
load_dotenv(override=True)

True

In [7]:
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-


## OpenAI compatible endpoints

`AsyncOpenAI` and `OpenAIChatCompletionsModel` can be used to connect to any OpenAI compatible endpoint.

If using OpenAI hosted models, we don't need `AsyncOpenAI` and `OpenAIChatCompletionsModel`, we can use `OpenAI` directly as usual.

In [8]:
ollama_api_key = os.getenv('OLLAMA_API_KEY')
ollama_url = os.getenv('OLLAMA_BASE_URL')


if ollama_api_key:
    print(f"Ollama API Key exists and begins {ollama_api_key[:8]}")
else:
    print("Ollama API Key not set")
if ollama_url:
    print(f"Ollama URL exists and is {ollama_url}")
else:
    print("Ollama URL not set")

Ollama API Key exists and begins ollama
Ollama URL exists and is http://localhost:11434/v1


In [6]:
# Models

model_gpt = os.getenv('MODEL_GPT5_NANO')
model_phi = os.getenv('MODEL_PHI4_14B')
model_llama = os.getenv('MODEL_LLAMA3_8B')
model_gpt_ollama = os.getenv('MODEL_GPTOSS_20B')

print(f"GPT Models: \
        \n- OpenAI: {model_gpt}\
        \n- Ollama: {model_gpt_ollama}")
print(f"Phi Model: {model_phi}")
print(f"Llama Model: {model_llama}")

GPT Models:         
- OpenAI: gpt-5-nano        
- Ollama: gpt-oss
Phi Model: phi4
Llama Model: llama3.1


In [9]:
gpt_ollama_client = AsyncOpenAI(base_url=ollama_url, api_key=ollama_api_key)
phi_client = AsyncOpenAI(base_url=ollama_url, api_key=ollama_api_key)
llama_client = AsyncOpenAI(base_url=ollama_url, api_key=ollama_api_key)

In [10]:
gpt_ollama_model = OpenAIChatCompletionsModel(model=model_gpt_ollama, openai_client=gpt_ollama_client)
phi_model = OpenAIChatCompletionsModel(model=model_phi, openai_client=phi_client)
llama_model = OpenAIChatCompletionsModel(model=model_llama, openai_client=llama_client)

# Multimodel Integration

We will build an agentic system that composes multiple cold sales emails using multiple models and sends the selected one using provided tools.

**Workflow:**

Start -> Draft Emails (multiple models) -> Select Best Email -> Format Email (HTML) -> Send Email

<br />

**Hierarchical Diagram:**

```mermaid
Sales Manager Agent
  ├── Sales Agents (multiple models)
  └── Emailer Agent
        ├── Subject Writer Agent
        ├── HTML Converter Agent
        └── Send Email Function
```

## Tools

We will use the following tools to compose and send emails:

1. **Sales Agents:** each agent uses a different model to compose a cold sales email based on agent's style.
2. **Custom Function:** Send email already converted to HTML. 
3. **Subject Writer Agent**: an agent to write email subject lines.
4. **HTML Converter Agent**: an agent to convert plain text email to HTML format.


In [12]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [13]:
sales_agent1 = Agent(name="Phi Sales Agent", instructions=instructions1, model=phi_model)
sales_agent2 =  Agent(name="Llama Sales Agent", instructions=instructions2, model=llama_model)
sales_agent3  = Agent(name="GPT-OSS Sales Agent",instructions=instructions3,model=gpt_ollama_model)

In [14]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

In [15]:
email = os.getenv("EMAIL")

In [16]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email(email)
    to_email = To(email)
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [17]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-5-nano")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-5-nano")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [18]:
email_tools = [subject_tool, html_tool, send_html_email]

## Handoff

**Emailer Agent:** An agent that formats an email and sends it, using the above tools.

In [20]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model="gpt-5-nano",
    handoff_description="Convert an email to HTML and send it")

In [21]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]

## Orchestration

**Sales Manager Agent:** An agent to manage email drafting and best email selection. Then, delegates sending the best email to the **Emailer Agent** agent for appropriate task.

In [22]:
sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-5-nano")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

## Check out the trace

We can see the full trace (https://platform.openai.com/traces) of the multimodel integration system in action below:

![Multimodel Integration Trace](../img/multimodel-integration-trace.png)

# Agentic Guardrails

Generally, we use guardrails to enforce input and output formats. They are placed at the beginning and/or end of the workflow to ensure data integrity. 

We will demonstrate input guardrails on Agent and tool functions.

## Structured Outputs

For structured outputs, we can use **Agentic Guardrails** to ensure the output adheres to a specific schema.

We specify this schema with Pydantic objects and ensure that the output from the model is always structured as we expect, and not a free-form text. 

We provide the schema during the agent initialization on the `output_schema` parameter.

In [28]:
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

guardrail_agent = Agent( 
    name="Name check",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput,
    model=model_gpt
)

## Function as guardrail

Use `@input_guardrail` decorator to enforce input guardrails on tool functions. This ensures that the inputs to the tool functions adhere to a specified schema.

Note that the tool function is defined as a coroutine (`async def`) to support asynchronous execution.

We must return a `GuardrailFunctionOutput` object from the tool function to comply with the expected output format. When the tool is called, the inputs are validated against the schema defined in the decorator. If the inputs do not conform to the schema, an error is raised, thus failing the guardrail check. This error is captured in `tripwire_triggered` attribute of the `GuardrailFunctionOutput`.



In [29]:
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output},tripwire_triggered=is_name_in_message)

For output guardrails, we can repeat the same process but instead but instead of providing a message to the guardrail function, we provide the output from the model.

```python
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    # Guardrail logic here
    ...
```

## Agent using guardrail

In [30]:
careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model=model_gpt,
    input_guardrails=[guardrail_against_name]
    )

### Guardrail Failure Path

In [31]:
message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

**Check out the trace**

![Guardrail Failure Trace](../../img/guardrail-failure-trace.png)

We see that the guardrail was triggered when the user input contained a personal name ("Alice"). The agent was able to identify this and respond appropriately, ensuring compliance with the guardrail.

<div  style="text-align: center; ">
<p><strong>Top level tripwire triggered:</strong></p>
<img src="../img/guardrail-tripwire-triggered-traces.png" alt="Top level tripwire" class="box" style="width: 400px; border: 2px solid gray;"/>
</div>

<div style="text-align: center;">
<p><strong>Function level tripwire status:</strong></p> 
<img src="../img/guardrail-against-name-true.png" alt="Function level tripwire" class="box" style="width: 400px; border: 2px solid gray;"/>
</div>

<div style="text-align: center;">
<p><strong>Response for Guardrail Function:</strong></p>
<img src="../img/guardrail-response.png" alt="Response for Guardrail Function" class="box" style="width: 400px; border: 2px solid gray;"/>
</div>

### Guardrail Happy Path

In [ ]:
message = "Send out a cold sales email addressed to Dear CEO from Head of Business Development"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

When the user input does not contain any personal names, the guardrail is not triggered, and the agent proceeds with the task as expected.

![Guardrail Happy Path Trace](../../img/guardrail-success-trace.png)

<div style="text-align: center;">
<p><strong>Function level tripwire status: <em>NOT</em> triggered</strong></p>
<img src="../img/guardrail-against-name-false.png" alt="Function level tripwire" class="box" style="width: 400px; border: 2px solid gray;"/>
</div>